In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle


# doc : https://colab.research.google.com/notebooks/io.ipynb#scrollTo=1w7lrGcW08Ds

# query ref : https://developers.google.com/drive/api/v3/reference/query-ref
# search terms : https://developers.google.com/drive/api/v3/ref-search-terms
# mime types : https://developers.google.com/drive/api/v3/mime-types

# files ref : https://developers.google.com/drive/api/v3/reference/files


In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
def get_folder_size(folder_name):

  # return id with id or title
  def get_id(**kwargs): # title or id + value
    for arg, value in kwargs.items():
      if arg == 'id':
        id = value
      elif arg == 'title':
        folder = drive.ListFile({"q": f"mimeType='application/vnd.google-apps.folder' and title = '{value}' and trashed=false"}).GetList()
        id = folder[0]['id']
    return id

  # mains vars
  id = get_id(title=folder_name)
  total_items = 0
  total_size = 0

  # folder iteration function
  def iterate_over_folder(folder_id):
    folder = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    for item in folder:
      #
      if item['mimeType'] != 'application/vnd.google-apps.folder':
        nonlocal total_size, total_items # --> referring to the main vars
        total_size += int(item['fileSize'])
        total_items += 1
      else:
        iterate_over_folder(item['id']) # --> RECURSION !! :D

  # root request
  base = drive.ListFile({'q': f"'{id}' in parents and trashed=false"}).GetList()

  for item in base:
    if item['mimeType'] != 'application/vnd.google-apps.folder':
      total_size += int(item['fileSize'])
      total_items += 1
    elif item['mimeType'] == 'application/vnd.google-apps.folder':
      iterate_over_folder(item['id'])

  # formating
  if total_size > 1000000000:
    total_size = f"{round(total_size/1000000000, 2)} Gb"
  else:
    total_size = f"{round(total_size/1000000, 2)} Mb"

  total_items = f"{total_items} items"

  return total_size, total_items

# checker la différence entre filesize et quotaBytesUsed
# 2005 : taille 311mb en filesize, 1.26Gb en quotaBytesUsed
# révision ?

print(get_folder_size('2005'))

('326.33 Mb', '411 items')
